In [ ]:
!pip install transformers

     |████████████████████████████████| 675kB 15.1MB/s 
     |████████████████████████████████| 890kB 58.4MB/s 
     |████████████████████████████████| 3.8MB 45.1MB/s 
     |████████████████████████████████| 1.1MB 56.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=0002c4671854c866147c96b627e223843d38f8dee62633fed64e9f39b52c9cff
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import os
import random
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from google.colab import drive
import textwrap
import progressbar
import keras
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import json

Using TensorFlow backend.


In [ ]:
from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig
from transformers import XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig
from transformers import XLMForSequenceClassification, XLMTokenizer, XLMConfig
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig

MODEL_CLASSES = {
    'bert': (BertForSequenceClassification, BertTokenizer, BertConfig),
    'xlnet': (XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig),
    'xlm': (XLMForSequenceClassification, XLMTokenizer, XLMConfig),
    'roberta': (RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig),
    'distilbert': (DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig)}

model_type = 'roberta' ###--> CHANGE WHAT MODEL YOU WANT HERE!!! <--###
model_class, tokenizer_class, config_class = MODEL_CLASSES[model_type]
model_name = 'roberta-base'

In [ ]:
drive.mount("/content/Drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/Drive


In [ ]:
output_dir = "/content/Drive/My Drive/RoBERTa_right_model2/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RobertaForSequenceClassification.from_pretrained(output_dir, output_hidden_states=True)
tokenizer = tokenizer_class.from_pretrained(output_dir)
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
df = pd.read_csv('/content/Drive/My Drive/LNLP/dataset.csv')
train_set = df.query(" split=='train' ")
test_set = df.query(" split=='test' ")
validation_set = df.query(" split=='dev' ")

In [ ]:
def att_masking(input_ids):
  attention_masks = []
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)
  return attention_masks

In [ ]:
def grouped_input_ids(all_toks):
  splitted_toks = []
  l=0
  r=510
  while(l<len(all_toks)):
    splitted_toks.append(all_toks[l:min(r,len(all_toks))])
    l+=410
    r+=410

  CLS = tokenizer.cls_token
  SEP = tokenizer.sep_token
  e_sents = []
  for l_t in splitted_toks:
    l_t = [CLS] + l_t + [SEP]
    encoded_sent = tokenizer.convert_tokens_to_ids(l_t)
    e_sents.append(encoded_sent)

  e_sents = pad_sequences(e_sents, maxlen=512, value=0, dtype="long", padding="post")
  att_masks = att_masking(e_sents)
  return e_sents, att_masks

In [ ]:
def get_output_for_one_vec(input_id, att_mask):
  input_ids = torch.tensor(input_id)
  att_masks = torch.tensor(att_mask)
  input_ids = input_ids.unsqueeze(0)
  att_masks = att_masks.unsqueeze(0)
  model.eval()
  input_ids = input_ids.to(device)
  att_masks = att_masks.to(device)
  with torch.no_grad():
      logits, encoded_layers = model(input_ids=input_ids, token_type_ids=None, attention_mask=att_masks)

  #vec = encoded_layers[12][0][0]
  #vec = vec.detach().cpu().numpy()
  #return vec
  logits = logits.detach().cpu().numpy()
  label = np.argmax(logits, axis=1).flatten()

  return label

In [ ]:
def generate_np_files_for_emb(dataf, tokenizer):
  all_docs = []
  ####
  my_preds = []
  ####
  for i in progressbar.progressbar(range(len(dataf['text']))):
    text = dataf['text'].iloc[i]
    toks = tokenizer.tokenize(text, add_prefix_space=True)
    if(len(toks) > 10000):
      toks = toks[len(toks)-10000:]

    splitted_input_ids, splitted_att_masks = grouped_input_ids(toks)

    vecs = []
    for index,ii in enumerate(splitted_input_ids):
      vecs.append(get_output_for_one_vec(ii, splitted_att_masks[index]))
 
    lval = 0.0
    for i in vecs:
      lval+=i

    if(lval/len(vecs) > 0.5):
      my_preds.append(1)
    else:
      my_preds.append(0)

  return my_preds
  #   one_doc = np.asarray(vecs)
  #   all_docs.append(one_doc)

  # all_docs = np.asarray(all_docs)
  # return all_docs

In [ ]:
preds_dev = generate_np_files_for_emb(validation_set, tokenizer)
dev_labels = validation_set['label'].to_list()
print(len(preds_dev))
correct=0
for index in range(len(preds_dev)):
  if(preds_dev[index] == dev_labels[index]):
    correct+=1

print(correct/len(preds_dev))
#np.save("/content/Drive/My Drive/LNLP/Hierarchical/BERT/BERT_dev.npy", vecs_dev)

100% (994 of 994) |######################| Elapsed Time: 0:08:23 Time:  0:08:23


994
0.6348088531187123


In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import textwrap
import progressbar
import keras
from keras.preprocessing.sequence import pad_sequences


def metrics_calculator(preds, test_labels):
    cm = confusion_matrix(test_labels, preds)
    TP = []
    FP = []
    FN = []
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[i][j]

        FN.append(summ)
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[j][i]

        FP.append(summ)
    for i in range(0,2):
        TP.append(cm[i][i])
    precision = []
    recall = []
    for i in range(0,2):
        precision.append(TP[i]/(TP[i] + FP[i]))
        recall.append(TP[i]/(TP[i] + FN[i]))

    macro_precision = sum(precision)/2
    macro_recall = sum(recall)/2
    micro_precision = sum(TP)/(sum(TP) + sum(FP))
    micro_recall = sum(TP)/(sum(TP) + sum(FN))
    micro_f1 = (2*micro_precision*micro_recall)/(micro_precision + micro_recall)
    macro_f1 = (2*macro_precision*macro_recall)/(macro_precision + macro_recall)
    return macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1



In [ ]:
metrics_calculator(preds_dev, dev_labels)

(0.6942946832843206,
 0.6348088531187123,
 0.663220583725073,
 0.6348088531187123,
 0.6348088531187123,
 0.6348088531187123)

In [ ]:
preds_test = generate_np_files_for_emb(test_set, tokenizer)
test_labels = test_set['label'].to_list()
print(len(preds_test))
correct=0
for index in range(len(preds_test)):
  if(preds_test[index] == test_labels[index]):
    correct+=1

print(correct/len(preds_test))
#np.save("/content/Drive/My Drive/LNLP/Hierarchical/BERT/BERT_dev.npy", vecs_dev)

100% (1517 of 1517) |####################| Elapsed Time: 0:13:01 Time:  0:13:01


1517
0.6242584047462096


In [ ]:
metrics_calculator(preds_test, test_labels)

(0.6820188702068465,
 0.6255462272513949,
 0.6525630456204138,
 0.6242584047462096,
 0.6242584047462096,
 0.6242584047462096)